# Data Prepocessing

In [2]:
import numpy as np
import pandas as pd
import csv
from sklearn import preprocessing
from sklearn.decomposition import PCA

## Bank Dataset

In [197]:
# Read data from file

with open('bank/bank-full.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    df = list(spamreader)
    df = np.asarray(df)
    df = pd.DataFrame(data=df[1:], columns=df[0])
    

In [198]:
# Inspecting the unknown values in the dataset

jobs = df['job']
unknown_jobs = 0
for job in jobs:
    if job == 'unknown':
        unknown_jobs = unknown_jobs + 1
        
educations = df['education']
unknown_education = 0
for education in educations:
    if education == 'unknown':
        unknown_education = unknown_education + 1

defaults = df['default']
unknown_default = 0
for default in defaults:
    if default == 'unknown':
        unknown_default = unknown_default + 1

contacts = df['contact']
unknown_contact = 0
for contact in contacts:
    if contact == 'unknown':
        unknown_contact = unknown_contact + 1
    
pdays = df['pdays']
unknown_pday = 0
for pday in pdays:
    if pday == '-1':      # Treating never contacted as unknown
        unknown_pday = unknown_pday + 1

pouts = df['poutcome']
unknown_pouts = 0
for pout in pouts:
    if pout == 'unknown':
        unknown_pouts = unknown_pouts + 1
    

print("Unknown values\n")
print("Unknown jobs :", unknown_jobs)
print("Unknown education :", unknown_education)
print("Unknown contacts :", unknown_contact)
print("Unknown pdays :", unknown_pday)
print("Unknown pouts :", unknown_pouts)



Unknown values

('Unknown jobs :', 288)
('Unknown education :', 1857)
('Unknown contacts :', 13020)
('Unknown pdays :', 36954)
('Unknown pouts :', 36959)


### Observations from Data Set

- Total number of rows: 45211
- Total number of attributes: 16 + output attribute (y)
- Yes samples - 5289
- No samples - 39922
- The number of yes samples are much smaller than the number of no samples indicating that the data set is highly skewed.

- Unknown values :-
    - job - 288
    - education - 1857
    - contact - 13020
    - pdays - 36954
    - poutcome - 36959
- A very large number of rows have pdays = -1 and poutcome = unknown. These represent clients that have not been contacted in the past

In [199]:
df = df.drop('duration', axis=1)             # Removing duration attribute

# Separating columns as numerical and categorical
numeric = ['age', 'balance', 'campaign', 'pdays', 'previous']
categorical = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'day', 'month', 'poutcome', 'y']


In [200]:
# Substituting pdays = 999 for pdays = -1
pdays = df['pdays']
pdays = pdays.replace('-1', '999')
df['pdays'] = pdays


# Substituting pdays = 999 and poutcome = failure for pdays = -1 and poutcome = unknown
# prev_list = ['pdays', 'poutcome']
# prev_df = df[prev_list]
# prev_df = prev_df.replace({'pdays':'-1', 'poutcome':'unknown'}, {'pdays':'999', 'poutcome':'failure'})
# df[prev_list] = prev_df


In [201]:
# Taking care of categorical values

# factorize assigns unique numeric values to each category
for c in categorical:
    df[c] = pd.factorize(df[c])[0]
    
# Use dummy attributes for categorical values
# df = pd.get_dummies(df, columns=categorical, prefix=categorical)
# df['y'] = pd.factorize(df['y'])

df[numeric] = df[numeric].astype('float')
df[categorical] = df[categorical].astype('category')

# df['y'] = df['y'].astype(int) # Converting otuput variable to integers


In [202]:
# Normalizing numerical values

scaler = preprocessing.StandardScaler()
rescaled_df = scaler.fit_transform(df[numeric])
df[numeric] = pd.DataFrame(rescaled_df, columns=numeric)

In [203]:
# Principal Component Analysis

y_values = df.loc[:, ['y']].values
df.drop(columns=['y'], axis=1, inplace=True)

pca = PCA()
#pca = PCA(n_components=10)
components = pca.fit_transform(df)
var_ratios = pca.explained_variance_ratio_
#rint(var_ratios)

# Calculating the number of attributes for 80% variance
s=0
count = 0
for ratio in var_ratios:
    s = s + ratio
    count = count + 1
    if s >= 0.8:
        break
print(count)

# Calculating the number of attributes for 95% variance
s=0
count1 = 0
for ratio in var_ratios:
    s = s + ratio
    count1 = count1+ 1
    if s >= 0.95:
        break
print(count1)

#df = pd.DataFrame(data=components)
df['y'] = y_values

2
5


### Observations from PCA

- There were 15 attributes (after removing duration) subjected to PCA
- 80% of the variance can be captured by 2 attributes after PCA
- 95% of the variance can be captured by 5 attributes after PCA

In [204]:
df


,0,1,2,3,4,5,6,7,8,9,y
0,-13.159835,-3.055055,-2.923286,-0.238970,0.996557,0.612685,-0.422731,-1.430510,0.237197,-0.462948,0
1,-13.171926,-2.805815,-1.856462,-0.242773,-0.014284,-0.101964,-0.718199,-0.246908,0.125590,0.501621,0
2,-13.175661,-2.546699,-0.904643,-0.221502,-0.725452,-0.051247,-0.610428,0.337272,0.162289,-0.614738,0
3,-13.186564,-2.223374,0.130993,-0.306661,0.925885,-0.078233,-0.731552,0.416416,0.134504,-0.332192,0
4,-13.187251,-1.940089,1.099114,-0.287128,-0.411237,-0.236615,-0.717435,0.788451,0.099766,0.541178,0
5,-13.161028,-3.106410,-2.907778,-0.172272,-0.927130,0.313955,-0.443188,-0.368621,0.191121,-0.685979,0
6,-13.159982,-3.119722,-2.902897,-0.129957,-1.594994,0.465704,-0.371055,-0.036598,0.156207,0.216878,0
7,-13.170490,-2.523582,-0.989784,-0.153157,-0.837104,0.361975,-0.454293,-1.009241,0.146274,1.306233,0
8,-13.202763,-1.644420,2.138288,-0.410963,1.973149,-0.852425,-1.120188,0.317797,0.111537,-0.135824,0
9,-13.171623,-2.803745,-1.859552,-0.233100,-0.024198,0.060649,-0.648496,-0.139655,0.125706,0.497253,0


In [205]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 11 columns):
0    45211 non-null float64
1    45211 non-null float64
2    45211 non-null float64
3    45211 non-null float64
4    45211 non-null float64
5    45211 non-null float64
6    45211 non-null float64
7    45211 non-null float64
8    45211 non-null float64
9    45211 non-null float64
y    45211 non-null int64
dtypes: float64(10), int64(1)
memory usage: 3.8 MB


In [206]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,y
count,4.521100e+04,4.521100e+04,4.521100e+04,4.521100e+04,4.521100e+04,4.521100e+04,4.521100e+04,4.521100e+04,4.521100e+04,4.521100e+04,45211.000000
mean,8.769571e-15,-1.695645e-15,1.232382e-15,-3.272888e-16,-3.288849e-17,1.089117e-16,8.060910e-17,-2.363831e-16,8.081538e-16,4.918663e-18,0.116985
std,8.365259e+00,3.173590e+00,3.104133e+00,1.229745e+00,1.087337e+00,9.997448e-01,9.864658e-01,8.428928e-01,6.883861e-01,6.774870e-01,0.321406
min,-1.322596e+01,-4.894689e+00,-6.780570e+00,-2.839595e+00,-2.603480e+00,-1.415291e+01,-1.601422e+00,-3.658642e+00,-2.309955e+00,-9.862233e-01,0.000000
25%,-7.180119e+00,-2.383164e+00,-2.608297e+00,-5.730371e-01,-8.288947e-01,-4.414418e-01,-5.292662e-01,-5.415711e-01,7.066182e-03,-5.348553e-01,0.000000
50%,-9.993422e-01,-1.005309e+00,-3.946397e-02,-3.305542e-01,-1.604116e-01,-4.290390e-02,-2.232861e-01,-2.747559e-02,1.247753e-01,-2.623658e-01,0.000000
75%,7.285694e+00,1.492574e+00,2.609055e+00,-1.206442e-01,6.919513e-01,2.947590e-01,2.260162e-01,4.743622e-01,2.082446e-01,3.588609e-01,0.000000
max,1.738375e+01,1.066274e+01,8.462027e+00,8.274590e+01,1.240621e+01,2.789238e+01,1.798276e+01,8.226162e+00,8.254661e+01,2.823029e+00,1.000000


In [207]:
#Saving as pickle file

df.to_pickle('data/bank_data_factorize_1.pkl')    # using factorization and replacing pdays only
#df.to_pickle('data/bank_data_factorize_2.pkl')    # using factorization and replacing pdays and poutcome

#df.to_pickle('data/bank_data_factorize_1_pca.pkl')    # using factorization and replacing pdays only and performing PCA
#df.to_pickle('data/bank_data_factorize_2_pca.pkl')    # using factorization and replacing pdays and poutcome and performing PCA

### Bank Additional Dataset

In [24]:
# Reading data from file

with open('bank-additional/bank-additional-full.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    df = list(spamreader)
    df = np.asarray(df)
    df = pd.DataFrame(data=df[1:], columns=df[0])

In [25]:
# Inspecting the unknown values in the dataset

jobs = df['job']
unknown_jobs = 0
for job in jobs:
    if job == 'unknown':
        unknown_jobs = unknown_jobs + 1
        
maritals = df['marital']
unknown_maritals = 0
for marital in maritals:
    if marital == 'unknown':
        unknown_maritals = unknown_maritals + 1

educations = df['education']
unknown_education = 0
for education in educations:
    if education == 'unknown':
        unknown_education = unknown_education + 1

defaults = df['default']
unknown_default = 0
for default in defaults:
    if default == 'unknown':
        unknown_default = unknown_default + 1

housing = df['housing']
unknown_housing = 0
for house in housing:
    if house == 'unknown':
        unknown_housing = unknown_housing + 1
    
loans = df['loan']
unknown_loans = 0
for loan in loans:
    if loan == 'unknown':
        unknown_loans = unknown_loans + 1
    
pdays = df['pdays']
unknown_pday = 0
for pday in pdays:
    if pday == '999':      # Treating never contacted as unknown
        unknown_pday = unknown_pday + 1

pouts = df['poutcome']
unknown_pouts = 0
for pout in pouts:
    if pout == 'nonexistent':
        unknown_pouts = unknown_pouts + 1
    

print("Unknown values\n")
print("Unknown jobs :", unknown_jobs)
print("Unknown marital status :", unknown_maritals)
print("Unknown education :", unknown_education)
print("Unknown default :", unknown_default)
print("Unknown housing :", unknown_housing)
print("Unknown loans :", unknown_loans)
print("Unknown pdays :", unknown_pday)
print("Unknown pouts :", unknown_pouts)


Unknown values

('Unknown jobs :', 330)
('Unknown marital status :', 80)
('Unknown education :', 1731)
('Unknown default :', 8597)
('Unknown housing :', 990)
('Unknown loans :', 990)
('Unknown pdays :', 39673)
('Unknown pouts :', 35563)


### Observations from Data Set

- Total number of rows: 41188
- Total number of attributes: 19 + output attribute (y)
- Yes samples - 4640
- No samples - 36548

- The number of yes samples are much smaller than the number of no samples indicating that the data set is highly skewed.

- Unknown values :-
    - job - 288
    - education - 1857
    - contact - 13020
    - pdays - 36954
    - poutcome - 36959
- A very large number of rows have pdays = 999 and poutcome = nonexistent. These represent clients that have not been contacted in the past

In [26]:
df = df.drop('duration', axis=1)             # Removing duration attribute

numeric = ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
categorical = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [27]:
# Taking care of categorical values

# factorize assigns unique numeric values to each category
for c in categorical:
    df[c] = pd.factorize(df[c])[0]
    
# Use dummy attributes for categorical values
# df = pd.get_dummies(df, columns=categorical, prefix=categorical)
# df['y'] = pd.factorize(df['y'])

df[numeric] = df[numeric].astype('float')
df[categorical] = df[categorical].astype('category')

# df['y'] = df['y'].astype(int) # Converting otuput variable to integers

In [28]:
# Normalizing numerical values

scaler = preprocessing.StandardScaler()
rescaled_df = scaler.fit_transform(df[numeric])
df[numeric] = pd.DataFrame(rescaled_df, columns=numeric)

In [29]:
# Principal Component Analysis

y_values = df.loc[:, ['y']].values
df.drop(columns=['y'], axis=1, inplace=True)

#pca = PCA()
pca = PCA(n_components=10)
components = pca.fit_transform(df)
var_ratios = pca.explained_variance_ratio_
#rint(var_ratios)

# Calculating the number of attributes for 80% variance
s=0
count = 0
for ratio in var_ratios:
    s = s + ratio
    count = count + 1
    if s >= 0.8:
        break
print(count)

# Calculating the number of attributes for 95% variance
s=0
count1 = 0
for ratio in var_ratios:
    s = s + ratio
    count1 = count1+ 1
    if s >= 0.95:
        break
print(count1)

df = pd.DataFrame(data=components)
df['y'] = y_values

6
10


### Observations from PCA

- There were 18 attributes (after removing duration) subjected to PCA
- 80% of the variance can be captured by 6 attributes after PCA
- 95% of the variance can be captured by 10 attributes after PCA


In [30]:
df

,0,1,2,3,4,5,6,7,8,9,y
0,-5.085238,1.976573,1.192552,-0.215155,-2.003972,0.848267,1.607368,-0.085110,-0.451382,0.102161,no
1,-4.245930,0.958810,0.738160,-0.208968,-2.013441,0.868691,1.707951,-0.000635,-0.638437,0.060624,no
2,-4.239274,0.939066,0.595273,-0.055896,-1.964206,0.416226,0.453556,-0.795599,0.563273,-0.107539,no
3,-3.379089,-0.068760,0.134065,-0.036606,-1.974160,0.474288,0.637311,-0.663590,0.327532,-0.052078,no
4,-4.220810,0.969717,0.712080,-0.169690,-2.008423,0.826473,1.603210,-0.065391,-0.520651,0.086857,no
5,-3.569816,0.414489,-1.113143,0.100325,-2.003916,0.617640,1.020843,-0.409641,-0.105124,-0.048261,no
6,-2.653864,-0.550303,-1.523710,0.097162,-2.043962,0.868999,1.832070,0.121289,-0.910122,0.109335,no
7,-1.844656,-1.606542,-2.096716,0.210928,-2.004811,0.493633,0.779250,-0.525972,-0.011746,-0.097869,no
8,-1.651643,-2.102220,-0.944987,0.173770,-1.949430,0.052176,-0.447901,-1.280976,1.170616,0.526923,no
9,-4.247304,0.919201,0.499158,0.036175,-1.935292,0.134031,-0.389039,-1.299040,1.316189,0.557737,no


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
age               41188 non-null float64
job               41188 non-null category
marital           41188 non-null category
education         41188 non-null category
default           41188 non-null category
housing           41188 non-null category
loan              41188 non-null category
contact           41188 non-null category
month             41188 non-null category
day_of_week       41188 non-null category
campaign          41188 non-null float64
pdays             41188 non-null float64
previous          41188 non-null float64
poutcome          41188 non-null category
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: category(10), float64(9), object(1)
memory usage: 3.5+

In [22]:
df.describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04
mean,2.846557e-16,1.845337e-14,9.515119e-14,2.196991e-13,-1.851020e-13,-6.434047e-14,-6.842191e-14,-6.864230e-14,-1.181213e-13
std,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00
min,-2.209365e+00,-5.659220e-01,-5.149443e+00,-3.494943e-01,-2.216433e+00,-2.374889e+00,-2.224953e+00,-1.722351e+00,-2.815697e+00
25%,-7.699804e-01,-5.659220e-01,1.954139e-01,-3.494943e-01,-1.197935e+00,-8.649547e-01,-4.747909e-01,-1.312994e+00,-9.402810e-01
50%,-1.942267e-01,-2.049085e-01,1.954139e-01,-3.494943e-01,6.480923e-01,2.994571e-01,-2.803284e-01,7.124599e-01,3.316799e-01
75%,6.694038e-01,1.561049e-01,1.954139e-01,-3.494943e-01,8.390606e-01,7.227225e-01,8.864466e-01,7.724221e-01,8.451701e-01
max,5.563310e+00,1.928982e+01,1.954139e-01,1.379492e+01,8.390606e-01,2.058168e+00,2.939106e+00,8.208531e-01,8.451701e-01


In [31]:
#Saving as pickle file
df.to_pickle('data/bank_additional_data_factorize_pca.pkl')